In [1]:
import undetected_chromedriver as uc
import time 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import json
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from datetime import datetime
from threading import Timer

from bs4 import BeautifulSoup


class repeater : 
    def __init__(self,func ,parameters = [],secs = 0) :
        self.clock = []
        self.func = func 
        self.parameters = parameters 
        self.secs = secs
        self.return_values = []
    def start(self) : 
        self.return_values.append(self.func(*self.parameters))
        if self.clock.__len__() >0 : self.clock.pop()
        self.clock.append(Timer(self.secs , self.start))
        self.clock[-1].start()
    def stop(self) : 
        print("Canceled Clock")
        for i in self.clock : i.cancel()

class Selchrominum : 
    def __init__(self,urls = [],delay  = 10,callback = []):
        self.caps = DesiredCapabilities.CHROME
        self.caps['goog:loggingPrefs'] = {'performance': 'ALL'}
        self.driver = uc.Chrome(headless=False,use_subprocess=False,desired_capabilities=self.caps)
        self.urls = urls
        self.delay = max(5,delay) 
        self.callback = callback
        self.repeater = None
        self.done = False
    def start_get(self): 
        print("Called start_get")
        self.repeater = repeater(self.__get,[],self.delay)
        self.repeater.start()
    def close(self) : 
        self.repeater.stop()
        self.driver.close()
        self.driver.quit()
    def __get(self) : 
        print("Called __get")
        try : 
            url = self.urls.pop(0)
            self.done = False
        except : 
            self.done = True
            return 
        try : 
            self.__fetch(url,self.delay)
        except : 
            self.driver.close()
            self.driver.quit()
            self.driver = uc.Chrome(headless=False,use_subprocess=False,desired_capabilities=self.caps)
            self.__fetch(url,self.delay)
    def __fetch(self,url,delay) : 
        print('Start GET ',url)
        self.driver.get(url)
        time.sleep(delay)
        
        for func in self.callback : 
            func(self.driver)





class shopee_shops_crawler : 
    def __init__(self,domain = 'https://shopee.vn/',shops = [],custom_querries = [],custom_parameters = [],save_location = '',number_of_browser = 2) -> None:
        self.domain = domain 
        self.shops = shops 
        self.count_max = 24
        self.count = 0 
        self.save_location = save_location
        self.url_querries = '&'.join([f'{i}={j}' for i,j in zip(custom_querries,custom_parameters)])
        self.stack_max = 6
        self.browsers = [{
            'name' : i,
            'browser' : Selchrominum(),
            'size_of_stack' : 0,    
            'tasks' : [], #list of list function

        } for i in range(number_of_browser)]

        self.urls = []
    def wait_for_cooldown(self) : 
        time.sleep(1*60*60)
        self.count = 0 
    def get_min_browser(self) : 
        temp_list = [browser['size_of_stack'] for browser in self.browsers]
        return self.browsers[temp_list.index(min(temp_list))]
    
    def __fetch_source_page(self,driver : uc.Chrome) : 
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        
        for script in soup.find_all('script'):
            script.extract()
        modified_html = str(soup)
        print(modified_html) ##################

    def __fetch_shop_infor(self,driver : uc.Chrome) : 
        key = 'get_shop_base'

        browser_log = self.driver.get_log('performance') 
        events = [json.loads(entry['message'])['message'] for entry in browser_log]
        events = [event for event in events if 'Network.response' in event['method']]

        requestId = None
        for i in events : 
            try : 
                if key in i["params"]["response"]["url"] : 
                    requestId = i["params"]["requestId"]
                    break
            except : 
                pass
        #print('get_shop_base')
        print(self.driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': requestId})) ##################

    def __fetch_shop_goods(self,driver : uc.Chrome) : 
        key = 'rcmd_items'

        browser_log = self.driver.get_log('performance') 
        events = [json.loads(entry['message'])['message'] for entry in browser_log]
        events = [event for event in events if 'Network.response' in event['method']]

        requestId = None
        for i in events : 
            try : 
                if key in i["params"]["response"]["url"] : 
                    requestId = i["params"]["requestId"]
                    break
            except : 
                pass
        #print('rcmd_items')
        print(self.driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': requestId})) ##################

    def start_crawl(self) : 
        for shop in self.shops : 
            self.urls.append(self.domain + shop + '?page=0' + self.url_querries)



#se lam ke thua 


t = Selchrominum(urls=['https://shopee.vn/5s_official?page=0&sortBy=pop',
                       'https://shopee.vn/5s_official?page=1&sortBy=pop',
                       'https://shopee.vn/5s_official?page=2&sortBy=pop',
                       'https://shopee.vn/5s_official?page=3&sortBy=pop'],
                       fetch_urls=['rcm','rcm','rcm','rcm'])


t.start_get()

while (t.done == False) :
    time.sleep(10)
t.close()

Called start_get
Called __get
Start GET  https://shopee.vn/5s_official?page=0&sortBy=pop
rcm
{'base64Encoded': False, 'body': '{"error":0,"error_msg":"","data":{"key":"shop_page_category_tab_main_sec","total":197,"no_more":false,"items":[{"itemid":17300501627,"shopid":127217331,"name":"Áo Polo Nam 5S FASHION VIRTUAL, Chất Liệu PE Cool Mát, Nhẹ, Thoáng Khí, Thiết Kế Trẻ Trung, Lịch Lãm (APC23059)","label_ids":[1000262,1000258,1000259,1000255,1000263,1000254,1000256,1000257,1000264,1000265,1000266,1000269,1000270,1000267,1000272,1000271,1000274,1000273,1000276,1000277,1000278,1000275,1000279,1000908,1000927,1000260,1000268,1000956,1000984,1001035,1006487,1006856,1008793,1008941,1009113,1009763,1012191,1014184,700066642,1400066701,1400066702,700085533,1208581,1208620,1243613,1208670,1278629,700085579,700185064,700190078,700195073,700190079,1400170039,1400085529,700155008,1400175056,1278696,1263701,1268699,1428581,1428593,1433638,1428679,1428689,1428733,1400290091,1428749,1428753,1433755,1

In [9]:

caps = DesiredCapabilities.CHROME
#  #as per latest docs
caps['goog:loggingPrefs'] = {'performance': 'ALL'}
driver = uc.Chrome(headless=False,use_subprocess=False,desired_capabilities=caps)
print('Start GET')

driver.get('https://shopee.vn/sp.btw2?page=5&sortBy=pop')
#driver.save_screenshot('nowsecure.png')
time.sleep(10)
delay = 10 # seconds



def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

browser_log = driver.get_log('performance') 
events = [process_browser_log_entry(entry) for entry in browser_log]
events = [event for event in events if 'Network.response' in event['method']]

Start GET


InvalidArgumentException: Message: invalid argument
  (Session info: chrome=118.0.5993.88)
Stacktrace:
	GetHandleVerifier [0x00664DE3+43907]
	(No symbol) [0x005F0741]
	(No symbol) [0x004E32B0]
	(No symbol) [0x004D5BE4]
	(No symbol) [0x004D450F]
	(No symbol) [0x004D4AF8]
	(No symbol) [0x004E5193]
	(No symbol) [0x00545B61]
	(No symbol) [0x00532B5C]
	(No symbol) [0x005455CA]
	(No symbol) [0x00532956]
	(No symbol) [0x0050E17E]
	(No symbol) [0x0050F32D]
	GetHandleVerifier [0x00915AF9+2865305]
	GetHandleVerifier [0x0095E78B+3163435]
	GetHandleVerifier [0x00958441+3138017]
	GetHandleVerifier [0x006EE0F0+605840]
	(No symbol) [0x005FA64C]
	(No symbol) [0x005F6638]
	(No symbol) [0x005F675F]
	(No symbol) [0x005E8DB7]
	BaseThreadInitThunk [0x773AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D97C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77D97C3E+238]


In [3]:

driver.get('https://shopee.vn/sp.btw2?page=6&sortBy=pop')
#driver.save_screenshot('nowsecure.png')
time.sleep(10)
delay = 10 # seconds



def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

browser_log = driver.get_log('performance') 
events = [process_browser_log_entry(entry) for entry in browser_log]
events = [event for event in events if 'Network.response' in event['method']]



see_more = driver.find_element(By.XPATH,'//button[contains(text(),"See More")]')

footer = driver.find_element(By.TAG_NAME,'footer')
driver.execute_script("arguments[0].remove();", footer)

from selenium.webdriver.common.action_chains import ActionChains
#driver.execute_script("arguments[0].click();", see_more)
#actions = ActionChains(driver)
#actions.move_to_element(see_more).perform()

time.sleep(3)

InvalidArgumentException: Message: invalid argument: log type 'performance' not found
  (Session info: chrome=118.0.5993.88)
Stacktrace:
	GetHandleVerifier [0x003C4DE3+43907]
	(No symbol) [0x00350741]
	(No symbol) [0x002433ED]
	(No symbol) [0x0029E1ED]
	(No symbol) [0x00292B23]
	(No symbol) [0x0026E17E]
	(No symbol) [0x0026F32D]
	GetHandleVerifier [0x00675AF9+2865305]
	GetHandleVerifier [0x006BE78B+3163435]
	GetHandleVerifier [0x006B8441+3138017]
	GetHandleVerifier [0x0044E0F0+605840]
	(No symbol) [0x0035A64C]
	(No symbol) [0x00356638]
	(No symbol) [0x0035675F]
	(No symbol) [0x00348DB7]
	BaseThreadInitThunk [0x773AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D97C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77D97C3E+238]


In [33]:
for item in test:
  if 'rcm' in item['name'] : 
    print(item)

{'connectEnd': 5386.699999988079, 'connectStart': 5386.699999988079, 'decodedBodySize': 361073, 'deliveryType': '', 'domainLookupEnd': 5386.699999988079, 'domainLookupStart': 5386.699999988079, 'duration': 235.5, 'encodedBodySize': 38886, 'entryType': 'resource', 'fetchStart': 5386.699999988079, 'firstInterimResponseStart': 0, 'initiatorType': 'fetch', 'name': 'https://shopee.vn/api/v4/shop/rcmd_items?bundle=shop_page_category_tab_main&limit=30&offset=0&shop_id=40342563&sort_type=1&upstream=', 'nextHopProtocol': 'h2', 'redirectEnd': 0, 'redirectStart': 0, 'renderBlockingStatus': 'non-blocking', 'requestStart': 5388.300000011921, 'responseEnd': 5621.699999988079, 'responseStart': 5572.399999976158, 'responseStatus': 200, 'secureConnectionStart': 5386.699999988079, 'serverTiming': [], 'startTime': 5386.199999988079, 'toJSON': {}, 'transferSize': 39186, 'workerStart': 5386.5}


In [5]:
print(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': '18008.161'}))

{'base64Encoded': False, 'body': '{"error":0,"error_msg":"","data":{"key":"shop_page_category_tab_main_sec","total":182,"no_more":false,"items":[{"itemid":12037295017,"shopid":40342563,"name":"Quần Jogger Túi Hộp Unisex Nam Nữ 4 Màu Năng Động Cao Cấp VICERO","label_ids":[1000262,1000258,1000259,1000260,1000263,1000255,1000256,1000254,1000257,1000264,1000265,1000266,1000267,1000270,1000269,1000272,1000271,1000274,1000275,1000278,1000276,1000279,1000908,1000927,1000268,1000273,1000277,1000984,1001035,1006487,1006856,1008793,1009763,1012191,1014184,1014267,1115185,700066642,700085482,1400066702,1400095073,700085579,1213576,1213632,1243613,700150015,700155008,1208670,1278591,1278592,1268596,1268599,1263605,1278629,700185064,700190078,700195073,700190079,1400170039,1400175056,1400170062,1278696,1263701,1268699,1428581,1428593,1433638,1428664,1428679,1428689,700305069,1428733,1400290091,1428749,1428753,1523584,1508583,1428713,298418372,1001044,1011387,700000280,700000290,844931086908638,8220

In [7]:
rID = None
for i in events : 
    try : 
        if 'rcm' in i["params"]["response"]["url"] : 
            print(i["params"]["requestId"])
            rID = i["params"]["requestId"]
    except : 
        pass

1948.158


In [8]:
print(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': rID}))

{'base64Encoded': False, 'body': '{"error":0,"error_msg":"","data":{"key":"shop_page_category_tab_main_sec","total":182,"no_more":false,"items":[{"itemid":12037295017,"shopid":40342563,"name":"Quần Jogger Túi Hộp Unisex Nam Nữ 4 Màu Năng Động Cao Cấp VICERO","label_ids":[1000262,1000258,1000259,1000260,1000263,1000255,1000256,1000254,1000257,1000264,1000265,1000266,1000267,1000270,1000269,1000272,1000271,1000274,1000275,1000278,1000276,1000279,1000908,1000927,1000268,1000273,1000277,1000984,1001035,1006487,1006856,1008793,1009763,1012191,1014184,1014267,1115185,700066642,700085482,1400066702,1400095073,700085579,1213576,1213632,1243613,700150015,700155008,1208670,1278591,1278592,1268596,1268599,1263605,1278629,700185064,700190078,700195073,700190079,1400170039,1400175056,1400170062,1278696,1263701,1268699,1428581,1428593,1433638,1428664,1428679,1428689,700305069,1428733,1400290091,1428749,1428753,1523584,1508583,1428713,298418372,1001044,1011387,700000280,700000290,844931086908638,8220

In [38]:
print(events[100]["params"]["response"].keys())

KeyError: 'response'

In [41]:
driver.close()
driver.quit()

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00C34DE3+43907]
	(No symbol) [0x00BC0741]
	(No symbol) [0x00AB32B0]
	(No symbol) [0x00ADDCEF]
	(No symbol) [0x00B02A56]
	(No symbol) [0x00AFF2A1]
	(No symbol) [0x00AFEE2D]
	(No symbol) [0x00A9273E]
	(No symbol) [0x00A92C3E]
	(No symbol) [0x00A9306A]
	GetHandleVerifier [0x00EE5AF9+2865305]
	GetHandleVerifier [0x00F2E78B+3163435]
	GetHandleVerifier [0x00F28441+3138017]
	GetHandleVerifier [0x00CBE0F0+605840]
	(No symbol) [0x00BCA64C]
	(No symbol) [0x00A9246E]
	(No symbol) [0x00A91BF8]
	GetHandleVerifier [0x00F6614C+3391212]
	BaseThreadInitThunk [0x773AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D97C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77D97C3E+238]


In [15]:
from datetime import datetime
from threading import Timer


class repeater : 
    def __init__(self,func ,parameters = [],secs = 0) :
        self.clock = []
        self.func = func 
        self.parameters = parameters 
        self.secs = secs
        self.return_values = []
    def start(self) : 
        self.return_values.append(self.func(*self.parameters))
        if self.clock.__len__ >0 : self.clock.pop()
        self.clock.append(Timer(self.secs , self.start))
        self.clock[-1].start()
    def stop(self) : 
        for i in self.clock : i.cancel()
        
def hello_world(text : str = 'hello'):
    print(text)
    #...
a = repeater(hello_world,['hello'],2)
b = repeater(hello_world,['hi'],3)
a.start()
b.start()
time.sleep(10)
a.stop()
b.stop()

hello
hi
hello
hi
hello
hi
hello
hello
hi


In [16]:
l = [1,2,3,4]
l.pop()
l

[1, 2, 3]

In [86]:
import requests
from browsermobproxy import Server
import time

# Start the BrowserMob Proxy server
server = Server("F:/Shopee/browsermob-proxy/bin/browsermob-proxy.bat")
server.start()
proxy = server.create_proxy()

In [87]:
server.stop()

In [ ]:
uc.ChromeOptions()

In [17]:
a = [1,2,3]
b = [3,4,5]
for i,j in zip(range(len(a)),range(len(b))) : 
    print(i,j,a,b)
    a.pop()
    b.pop()

0 0 [1, 2, 3] [3, 4, 5]
1 1 [1, 2] [3, 4]
2 2 [1] [3]


In [20]:
a = [1,2,3]
b = [3,4,5]
len(tuple(zip(a,b)))

3

In [21]:
a = []
b = []
a.pop()

IndexError: pop from empty list

In [ ]:
b = [1,2,3,4,5,4]
print(b.)

In [7]:
a = [2,6,2,7,62,1,9] 
b = ['3','4','5']
'&'.join([f'{i}={j}' for i,j in zip(a,b)])

'2=3&6=4&2=5'

In [8]:
a.index(min(a))

5

In [12]:
input("Please press Enter: ")

''